In [1]:
import os
import os.path as op
import numpy as np
import pandas as pd
import sys
import boto3
from boto3 import session
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
locOld = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/austin-2010-2018-central/2018/" 
mode_choice_raw = pd.read_csv(locOld + 'final_trip_mode_choice_raw.csv')
utilities = pd.read_csv(locOld + 'final_trip_mode_choice_utilities.csv')

In [6]:
#head
mode_choice_raw.head()

,trip_id,0,1,2,3,4,5,6,7,8,...,370,371,372,373,374,375,376,377,378,trip_purpose
0,153537,0.0,0.0,0.0,0.0,1.0,-0.839923,-0.0,-0.052205,-0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4185,-0.0,-0.837,5.0
1,153541,0.0,0.0,0.0,0.0,1.0,-0.906883,-0.0,-0.048560,-0.016633,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4185,-0.0,-0.837,5.0
2,5426433,0.0,0.0,0.0,0.0,0.0,-0.358391,-0.0,-0.024156,-0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4185,-0.0,-0.837,5.0
3,5426437,0.0,0.0,0.0,0.0,0.0,-0.417468,-0.0,-0.038672,-0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4185,-0.0,-0.837,5.0
4,5426465,0.0,0.0,0.0,0.0,0.0,-0.403347,-0.0,-0.030302,-0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.4185,-0.0,-0.837,5.0


In [2]:
%%time
#Reading the files
loc = "https://beam-outputs.s3.amazonaws.com/pilates-outputs/sfbay-2010-base-20200206/activitysim/" 
trips = pd.read_csv(loc + 'final_trips.csv')
households = pd.read_csv(loc + 'final_households.csv')
persons = pd.read_csv(loc + 'final_persons.csv')
tours = pd.read_csv(loc +'final_tours.csv')
plans = pd.read_csv(loc +'final_plans.csv')
landuse = pd.read_csv(loc +'final_land_use.csv')
checkpoints = pd.read_csv(loc + 'final_checkpoints.csv')
joint_tour_participants = pd.read_csv(loc + 'final_joint_tour_participants.csv')
#specs = pd.read_csv(loc +'final_trip_mode_choice_specs.csv')
#tripModeChoice = pd.read_csv(loc +'final_trip_mode_choice_raw.csv')
#tripModeChoiceUtilities = pd.read_csv(loc +'final_trip_mode_choice_utilities.csv')

In [19]:
trips[trips['person_id']==100008]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
8513043,32802889,100008,2085387,4100361,shopping,1,True,2,shopping,485.0,512.0,14.737636,20.0,SHARED3FREE,1.022477
8513044,32802890,100008,2085387,4100361,shopping,2,True,2,shopping,644.0,485.0,NaN,20.0,SHARED3FREE,-0.069823
8513045,32802893,100008,2085387,4100361,shopping,1,False,1,Home,512.0,644.0,NaN,23.0,SHARED3FREE,0.090811
8513046,32802937,100008,2085387,4100367,work,1,True,1,work,311.0,512.0,NaN,6.0,DRIVEALONEFREE,-0.441274
8513047,32802941,100008,2085387,4100367,work,1,False,1,Home,512.0,311.0,NaN,18.0,SHARED2FREE,-1.740500


In [25]:
trips[trips['person_id']==1000104]

,trip_id,person_id,household_id,tour_id,primary_purpose,trip_num,outbound,trip_count,purpose,destination,origin,destination_logsum,depart,trip_mode,mode_choice_logsum
9612351,328034113,1000104,1098957,41004264,atwork,1,True,1,atwork,24.0,15.0,NaN,13.0,WALK,0.586553
9612352,328034117,1000104,1098957,41004264,atwork,1,False,1,Work,15.0,24.0,NaN,14.0,WALK,0.304286
9612353,328034425,1000104,1098957,41004303,work,1,True,1,work,15.0,106.0,NaN,7.0,WALK_LOC,-0.883453
9612354,328034429,1000104,1098957,41004303,work,1,False,1,Home,106.0,15.0,NaN,18.0,WALK_LOC,-1.036639


In [20]:
tours.head()

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,tdd,start,end,duration,composition,destination_logsum,tour_mode,mode_choice_logsum,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
0,39,0,work,1,1,1,1,mandatory,1,1440.0,1005.0,0,109.0,11.0,21.0,10.0,NaN,NaN,DRIVEALONEFREE,-1.215672,eat,NaN,0out_0in,work
49383,28,0,othmaint,1,1,1,1,non_mandatory,1,1005.0,1005.0,0,56.0,8.0,10.0,2.0,NaN,12.951899,SHARED2PAY,0.286401,NaN,NaN,0out_2in,othmaint
94580,4,0,eat,1,1,1,1,atwork,1,1441.0,1440.0,0,99.0,11.0,11.0,0.0,NaN,18.638677,WALK,0.315750,NaN,39.0,0out_0in,atwork
258335,74,1,shopping,1,1,1,1,non_mandatory,1,535.0,542.0,1681635,114.0,12.0,14.0,2.0,NaN,12.125930,DRIVEALONEFREE,-0.951092,NaN,NaN,0out_0in,shopping
258336,107,2,othdiscr,1,1,1,1,non_mandatory,1,560.0,542.0,1681635,150.0,15.0,20.0,5.0,NaN,13.552076,WALK_EXP,-0.831458,NaN,NaN,0out_0in,othdiscr


In [5]:
#Show all columns and rows
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [5]:
#Merge households and persons 
persons = persons.sort_values(by=['household_id'])
households = households.sort_values(by=['household_id'])
hhpersons = pd.merge(left=persons, right=households, how='left', on='household_id')

In [6]:
#Merge tours, households and persons
tours = tours.sort_values(by=['person_id'])
hhpersons = hhpersons.sort_values(by=['person_id'])
hhperTours = pd.merge(left=tours, right=hhpersons, how='left', on='person_id')

In [7]:
#Merge trips, tours, households and persons
trips = trips.sort_values(by=['person_id', 'tour_id'])
hhperTours = hhperTours.sort_values(by=['person_id','tour_id'])
tourTripsMerged = pd.merge(left=trips, right=hhperTours, how='left', on=['person_id','tour_id'])

In [14]:
tours[tours['person_id'] == 100008]

,tour_id,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,number_of_participants,destination,origin,household_id,tdd,start,end,duration,composition,destination_logsum,tour_mode,mode_choice_logsum,atwork_subtour_frequency,parent_tour_id,stop_frequency,primary_purpose
3340095,4100367,100008,work,1,1,1,1,mandatory,1,311.0,512.0,2085387,31.0,6.0,18.0,12.0,NaN,NaN,SHARED2FREE,-1.729210,no_subtours,NaN,0out_0in,work
3389168,4100361,100008,shopping,1,1,1,1,non_mandatory,1,644.0,512.0,2085387,183.0,20.0,23.0,3.0,NaN,10.160524,SHARED3PAY,-0.954841,NaN,NaN,1out_0in,shopping


In [ ]:
#Merge trips,tours, households, and persons with land-use
landuse = landuse.sort_values(by=['TAZ'])
tourTripsMerged = tourTripsMerged.sort_values(by=['TAZ_x'])
landuseTripsMerged = pd.merge(tourTripsMerged, landuse, how='left', left_on='TAZ_x', right_on='TAZ')

In [ ]:
#Merge trips, tours, households, persons and final_trip_mode_choice_raw
landuseTripsMerged = landuseTripsMerged.sort_values(by=['trip_id'])
tripModeChoice = tripModeChoice.sort_values(by=['trip_id'])
ActSimMerged = pd.merge(left=landuseTripsMerged, right=tripModeChoice, how='left', on=['trip_id'])

In [ ]:
specs.Expression_id = specs.Expression_id .astype(str)
specs.rename(columns = {'Expression_id':'trip_mode'}, inplace = True)
specs = specs.set_index('trip_mode')

In [ ]:
specsT = specs.transpose()

In [ ]:
ActSimSpec = pd.merge(ActSimMerged, specsT, left_on = 'trip_mode', right_index=True, how = 'left')

In [ ]:
#ActSimSpec.to_csv('C:/Shared-Work/Data/CleanData/Final_ActSimMerged.csv')